In [ ]:
import os
#setting up plaidml as backend to use my local gpu in Macbook pro
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend" 


In [ ]:
os.chdir('/Users/nasifimtiaz/Desktop/Caltech256-project')
print(os.getcwd())

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

#images are in different shapes. needs to square them.
def shrink_square(img, shrink_size, color_mode, fill_color):
    #all the images will be shrinked to shrink_size with any leftover filled with color_fill
    shrinked_img=img.thumbnail((shrink_size,shrink_size),Image.ANTIALIAS)
    
    #if some images were originally less than shrink_size, bring to shrink size by filling the leftover
    output=Image.new(color_mode,(shrink_size,shrink_size),fill_color)
    output.paste(img, (int((shrink_size - img.size[0]) / 2), int((shrink_size - img.size[1]) / 2)))
    
    return output


#turn images to np_array
def image_to_tensors(img):
    arr=np.array(img)
    return arr

        

In [ ]:
hm={}
path='./256_ObjectCategories'
folders=os.listdir(path)

for folder in folders:
    os.chdir(path+'/'+folder)
    image_paths=os.listdir()
    class_images=[]
    for i in image_paths:
        if i[-4:]=='.jpg':
            print("processing ",i)
            img=Image.open(i)
            shrinked_img=shrink_square(img, 128, 'RGB', 0)
            img_arr=image_to_tensors(shrinked_img)
            class_images.append(img_arr)
            img.close()
            shrinked_img.close()
    class_name= folder.split('.')[1]
    hm[class_name]=class_images
    os.chdir('../..')


In [21]:
print("total classes: ", len(hm))
sum=0
flawed=0
for k in hm.keys():
    sum+=len(hm[k])
    for i in hm[k]:
        if i.shape != (128,128,3):
            flawed+=1
print("total images: ",sum)
print("flawd images: ", flawed)

total classes:  257
total images:  30607
flawd images:  0
